**Requirements**

In [ ]:
!pip install telethon
!pip install hazm
!pip install xlsxwriter

# Persian messages cleaning and processing


In [ ]:
import regex
import re

class Preprocess:

  # This method search to find http links and replace them by '#لینک#'
  def find_urls(self, text):
    txt = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',"#لینک#", text)
    return  txt
  #  This method remove symbols that are not needed.
  def remove_punctuation(self,string):
    punctuation_pattern = re.compile(r"@|\*|\$|\%|\&|\)|\(|\}|\{|\[|\]|\(|/-|\+|(…)|_|[a-zA-Z]|:|;|=")
    string = re.sub(r"(؟)(؟)+","؟",re.sub(r'(!)(!)+',"!",string))
    return re.sub(r'( )( )+', r' ', punctuation_pattern.sub(r' ', string.replace("،", " ، ")))
  #  This method search to find emojies and remove them.
  def remove_emoji(self,string):
        emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
        return re.sub(r'( )( )+', r' ', emoji_pattern.sub(r'', string).replace("\n"," "))

# Get data from Telegram
To run this section, you need get id and hash from Telegram

Follow these steps:

1.  [Login to your Telegram account](https://my.telegram.org/) with the phone number of the developer account to use.
2.  Click under API Development tools.
3.  A Create new application window will appear. Fill in your application details. There is no need to enter any URL, and only the first two fields (App title and Short name) can currently be changed later.
4.  Click on Create application at the end. Remember that your API hash is secret and Telegram won’t let you revoke it. Don’t post it anywhere!

Then you must determine link a public group or channel in Telegram that you want to gather its messages.

In this notebook text of every messages have been saved in a excel file.

In [ ]:
import pytz
from telethon import TelegramClient
import datetime
from pytz import timezone
import xlsxwriter as xlwr

from hazm import *
import pandas as pd
from telethon.tl.functions.channels import GetParticipantsRequest
from telethon.tl.types import ChannelParticipantsSearch
from time import sleep

normalizer_hazm = Normalizer()
normalizer = Normalizer()
api_id = 0000000  # Insert your own id
api_hash = 'cccccccccccccccc' #Insert your own hash
client = TelegramClient('anon', api_id, api_hash)


def message_process(new_message):
    post= {}
    post['post_id'] = int(str(id) + str(new_message.id))
    post['message'] = new_message.text
    post['sender_username'] = new_message.username
    # post['sender_title'] = utils.get_display_name(sender)

    try:
        if new_message.views:
            post['view'] = new_message.views
        else:
            post['view'] = -1
    except KeyError:
        post['view'] = -1
    post['process'] = 'false'
    post['date_get'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    post['date_published'] = new_message.date.astimezone(timezone('Asia/Tehran')).strftime("%Y-%m-%d %H:%M:%S")



async def main():
    t_preprocess = Preprocess()
    # Getting information about yourself
    me = await client.get_me()
    print(me.stringify())
    username = me.username
    print(username)
    print(me.phone)
    group_list = [

    #      {"name":"Name of a public group",
    #        "link":"Link of a public group",
    #      "date":datetime.datetime.now()},

                        ]
    for group in group_list:
        message_texts = []
        message_ids = []
        message_users = []
        p_messages = []
        users_names = []
        users_id = []
        print(group['name'])
        excel_data_file = xlwr.Workbook("G:\master_matus\payan_name1\\2_implement1\\data\\" + group['name'] + ".xlsx")
        # excel_data_file = xlwr.Workbook("G:\\master_matus\\payan_name1\\2_implement1\\data\\" + 'borse_yar' + ".xlsx")
        # excel_data_file = pd.ExcelWriter("G:\\master_matus\\payan_name1\\2_implement1\\first_predict_model\\"+group['name']+".xlsx")
        previous_date = (group['date'].year, group['date'].month, group['date'].day)
        date_sheet = excel_data_file.add_worksheet(str(group['date'].year)+"-" + str(group['date'].month)+"-" +str(group['date'].day))
        date_sheet.write(0, 0, 'user_id')
        date_sheet.write(0, 1,  "user_name")
        date_sheet.write(0, 2, "name")
        date_sheet.write(0, 3, "message_id")
        date_sheet.write(0, 4, "date")
        date_sheet.write(0, 5, "reply_to")
        date_sheet.write(0, 6, "text")
        date_sheet.write(0, 7, "processed text")
        i = 1
        j = 1
        day = 0
        # all_participants = []
        # offset = 0
        # limit = 100
        # while True:
        #     participants = await client(GetParticipantsRequest(
        #                          group['name'], ChannelParticipantsSearch(''), offset, limit,
        #                           hash=0
        #       ))
        #     if not participants.users:
        #         break
        #     all_participants.extend(participants.users)
        #     offset += len(participants.users)
        # ids_list = []
        # active_status = []
        # fake_status = []
        # f_names = []
        # l_names = []
        # user_names_list = []
        # for member in all_participants:
        #     ids_list.append(member.id)
        #     active_status.append(~member.deleted)
        #     fake_status.append(member.fake)
        #     f_names.append(member.first_name)
        #     l_names.append(member.last_name)
        #     user_names_list.append(member.username)
        # pd.DataFrame({"id":ids_list,
        #               "active":active_status,
        #               "fake":fake_status,
        #                "first name":f_names,
        #                 "last name":l_names,
        #                  "username":user_names_list}).to_excel("users-"+group["name"]+".xlsx")
        async for message in client.iter_messages(group['link'], offset_date=group['date']):
            print(message)
            date_time = message.date
            new_timezone_timestamp = date_time.astimezone(timezone('Asia/Tehran'))
            if message.text is None or message.from_id is None:
                  continue
            j += 1
            message_date = (message.date.year, message.date.month, message.date.day)
            try:
                try:
                    reply_to = message.reply_to.reply_to_msg_id
                except AttributeError:
                    reply_to = None
                txt = normalizer.normalize(message.text)
                print(message)
                txt = t_preprocess.remove_punctuation(t_preprocess.find_urls(t_preprocess.remove_emoji(txt.replace("#"," #"))))
                user_id = None
                if message.media is None and message.from_id is not None:
                    try:
                        user_id = message.from_id.user_id
                    except Exception :
                        user_id = message.from_id.channel_id
                    try:
                        user = await client.get_entity(user_id)
                        if user.last_name is not None:
                            name = user.first_name + ' ' + user.last_name
                        else:
                            name = user.first_name
                    except AttributeError:
                        user = await client.get_entity(user_id)
                        name = user.title
                    except Exception:
                        name = "unknown"
                    if message_date == previous_date:

                            date_sheet.write(i, 0, user_id)
                            date_sheet.write(i, 1, user.username)
                            date_sheet.write(i, 2, name)
                            date_sheet.write(i, 3, message.id)
                            date_sheet.write(i,4,str(new_timezone_timestamp))
                            date_sheet.write(i,5,reply_to)
                            date_sheet.write(i, 6, message.text)
                            date_sheet.write(i, 7, txt)
                            i += 1
                    else:
                    # print(len(message_ids), len(message_texts), len(message_users), len(p_messages))
                    # pd.DataFrame({"id": message_ids,
                    #               'user_id':users_id,
                    #               "user_name": message_users,
                    #                'name':users_names,
                    #                "text": message_texts,
                    #               "processed text": p_messages}).to_excel(excel_data_file, sheet_name=date_sheet)
                    # print(i, ":?")
                    # message_texts = []
                    # message_ids = []
                    # message_users = []
                    # p_messages = []
                    # users_names = []
                    # users_id = []
                        txt = t_preprocess.remove_punctuation(
                        t_preprocess.find_urls(t_preprocess.remove_emoji(message.text)))
                        txt = normalizer.normalize(txt)
                        i = 1
                        previous_date = (message.date.year, message.date.month, message.date.day)
                    # date_sheet = str(message.date.year) + "-" + str(message.date.month) + "-" + str(message.date.day)
                        # Create new sheet to save messages that was sent at new date.
                        # Save inf;ormation of a message.
                        date_sheet = excel_data_file.add_worksheet(str(message.date.year)+"-" + str(message.date.month) + "-" + str(message.date.day))
                        date_sheet.write(0, 0, 'user_id')
                        date_sheet.write(0, 1,  "user_name")
                        date_sheet.write(0, 2, "name")
                        date_sheet.write(0, 3, "message_id")
                        date_sheet.write(0, 4, "date")
                        date_sheet.write(0, 5, "reply_to")
                        date_sheet.write(0, 6, "text")
                        date_sheet.write(0, 7, "processed text")

                        day += 1
                        date_sheet.write(i, 0, user_id)
                        date_sheet.write(i, 1, user.username)
                        date_sheet.write(i, 2, name)
                        date_sheet.write(i, 3, message.id)
                        date_sheet.write(i,4,str(new_timezone_timestamp))
                        date_sheet.write(i,5, reply_to)
                        date_sheet.write(i, 6, message.text)
                        date_sheet.write(i, 7, txt)

                        i += 1
                print(day)
                if day > 600:
                        print("300?")
                        break
                if len(txt) < 10:
                    continue

            except IndexError:
                print("**")
        excel_data_file.close()
from asyncio import run
run(main())

